## Imports

In [ ]:
import tensorflow as tf
import numpy as np
import sys
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Rescaling, Input
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from keras.losses import Loss, CosineSimilarity
from dataclasses import dataclass
from keras import backend as K
from sklearn.utils import shuffle
import math
from functools import partial
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, mean_absolute_error
# Load the TensorBoard notebook extension
%load_ext tensorboard
np.set_printoptions(threshold=sys.maxsize)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Dataset

In [ ]:
TRAIN_SET_RATE = 0.65
VALID_SET_RATE = 0.15
TEST_SET_RATE = 0.20

In [ ]:
@dataclass
class Dataset():
  name: str
  input_shape: object
  x_train: object
  y_train: object
  x_valid: object
  y_valid: object
  x_test: object
  y_test: object

## Load dataset

In [ ]:
x = np.load('data/images.npy')
y = np.load('data/labels.npy')
print(x.shape)
print(y.shape)

(18000, 150, 150)
(18000, 2)


## Prepare dataset

In [ ]:
if K.image_data_format() == 'channels_first':
  x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2])
  input_shape = (1, x.shape[1], x.shape[2])
else:
  x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
  input_shape = (x.shape[1], x.shape[2], 1)

x = x.astype('float32')
x, y = shuffle(x, y, random_state=42)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SET_RATE, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=VALID_SET_RATE, random_state=42)

print(f'Input shape: {input_shape}')
print(f'Train set shape: {x_train.shape}')
print(f'Validation set shape: {x_valid.shape}')
print(f'Test set shape: {x_test.shape}')

Input shape: (150, 150, 1)
Train set shape: (12240, 150, 150, 1)
Validation set shape: (2160, 150, 150, 1)
Test set shape: (3600, 150, 150, 1)


## Output transformations

In [ ]:
def decimal_representation_of(y):
  return y[:,0] + y[:,1] / 60

def cyclical_representation_of(y):
  decimal_y = decimal_representation_of(y)
  return np.array([np.sin(2*np.pi*decimal_y/12), np.cos(2*np.pi*decimal_y/12)])

def cyclical_representation_of_hours(hours):
  return np.array([np.sin(2*np.pi*hours/12), np.cos(2*np.pi*hours/12)])

def cyclical_representation_of_minutes(minutes):
  return np.array([np.sin(2*np.pi*minutes/60), np.cos(2*np.pi*minutes/60)])


def represented_in_range(number, interval_in_minutes):
  return [
          1 if number >= i and number < i + interval_in_minutes/60 else 0
          for i in np.arange(0, 12, interval_in_minutes / 60)
          ]

def grouped_in_classes(y, interval_in_minutes=30):
  decimal_y = decimal_representation_of(y)
  return np.array(
      [
       represented_in_range(yi, interval_in_minutes) 
       for yi in decimal_y
      ])


# Models

In [ ]:
BATCH_SIZE = 128
EPOCHS = 50
PATIENCE_IN_EPOCHS = 10

## Custom loss functions

In [ ]:
class DecimalTimesMeanLoss(Loss):

  def call(self, y_true, y_pred):
    return tf.reduce_mean(
      tf.math.minimum(
          tf.math.abs(y_true - y_pred),
          tf.math.abs(tf.math.minimum(y_true, y_pred) + 12 - tf.math.maximum(y_true, y_pred))
          )
      )
    
class MinutesMeanLoss(Loss):

  def call(self, y_true, y_pred):
    return tf.reduce_mean(
      tf.math.minimum(
          tf.math.abs(y_true - y_pred),
          tf.math.abs(tf.math.minimum(y_true, y_pred) + 60 - tf.math.maximum(y_true, y_pred))
          )
      )

class CyclicalTimesDistanceMeanLoss(Loss):

  def call(self, y_true, y_pred):
    loss = tf.reduce_mean(
        tf.math.sqrt(tf.reduce_sum(
            tf.math.square(y_true - y_pred), 
            axis=1
            ))
        )
    return loss

class CyclicalTimesMinutesMeanLoss(Loss):

  def call(self, y_true, y_pred):

    dot_product = tf.reduce_sum(tf.multiply(y_pred, y_true), axis=1)
    y_pred_norm = tf.norm(y_pred, axis=1)
    y_true_norm = tf.norm(y_true, axis=1)
    multiplied_norms = tf.multiply(y_pred_norm, y_true_norm)

    arccos = tf.math.acos(dot_product / multiplied_norms)
    arccos = tf.where(tf.math.is_nan(arccos), tf.zeros_like(arccos), arccos)

    return 60 * (arccos / (2*np.pi) )

class DecimalTimesCosineSimilarityLoss(Loss):

  def call(self, y_true, y_pred):
    y_cyclical_true = tf.map_fn(lambda x: [tf.math.sin(2* np.pi * x / 12), tf.math.cos(2* np.pi * x / 12)], y_true, dtype=[tf.float32, tf.float32])
    y_cyclical_pred = tf.map_fn(lambda x: [tf.math.sin(2* np.pi * x / 12), tf.math.cos(2* np.pi * x / 12)], y_pred, dtype=[tf.float32, tf.float32])

    cosine_loss = CosineSimilarity(axis=1)
    return cosine_loss(y_cyclical_true, y_cyclical_pred)


def adjusted_mae_numpy(a, b, max_value):
        
    return np.average(np.min(np.concatenate((np.abs(a - b), np.abs(np.min(np.concatenate((a, b), axis=1), axis=1) + max_value - np.max(np.concatenate((a, b), axis=1), axis=1)).reshape(-1, 1)), axis=1), axis=1))


## Regression CNN

In [42]:
def mean_minutes_loss_for_cyclical_time(y_true, y_pred):
  y_pred_unit_vectors = tf.map_fn(lambda x: x / tf.norm(x), y_pred)
  y_true_unit_vectors = tf.map_fn(lambda x: x / tf.norm(x), y_true)
  print(y_pred_unit_vectors)
  print(y_true_unit_vectors)
  minutes_losses = tf.map_fn(
      lambda i: 
      60 * 12 * tf.acos(tf.tensordot(y_pred_unit_vectors[i], y_true_unit_vectors[i], 1)) / (2 * tf.constant(np.pi)) , 
      tf.range(y_pred_unit_vectors.shape[0])
      )
  return tf.reduce_mean(minutes_losses)
  
def mean_minutes_loss_metric(y_true, y_pred):
    return tf.reduce_mean(
      tf.math.minimum(
          tf.math.abs(y_true - y_pred),
          tf.math.abs(tf.math.minimum(y_true, y_pred) + 12 - tf.math.maximum(y_true, y_pred))
          )
      ) * 60
    
def regression_cnn_1(loss, dataset, output_units, metric=None):
  DefaultConv2D = partial(Conv2D,kernel_size=3, activation='leaky_relu', padding="VALID")

  model = keras.models.Sequential([
    Rescaling(1./255, input_shape=dataset.input_shape),
    DefaultConv2D(filters=16, kernel_size=5),
    MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=32),
    DefaultConv2D(filters=32),
    MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    MaxPooling2D(pool_size=2),
    Dropout(0.4),
    Flatten(),
    
    Dense(units=512, activation='elu', kernel_initializer='he_normal'),
    Dense(units=512, activation='elu', kernel_initializer='he_normal'),
    Dense(units=output_units, activation='linear'),
  ])

  if metric != None:
    model.compile(optimizer='adam', loss=loss, metrics=[metric])
  else:
    model.compile(optimizer='adam', loss=loss)

  model.summary()
  return model

## Classification CNN

In [ ]:
def classification_cnn(dataset, classes, loss='categorical_crossentropy'):
  tf.random.set_seed(42)
  DefaultConv2D = partial(Conv2D,kernel_size=3, activation='leaky_relu', kernel_initializer='he_normal')

  model = keras.models.Sequential([
    Rescaling(1./255, input_shape=dataset.input_shape),
    DefaultConv2D(filters=16, kernel_size=5),
    MaxPooling2D(pool_size=(2,2)),
    DefaultConv2D(filters=32),
    DefaultConv2D(filters=32),
    MaxPooling2D(pool_size=(2,2)),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.5),
    Flatten(),
    Dense(units=64, activation='leaky_relu', kernel_initializer='he_normal'),
    Dense(units=64, activation='leaky_relu', kernel_initializer='he_normal'),
    # Dense(units=1024, activation='relu'),
    # Dropout(0.2),
    # Dense(units=512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    # Dense(units=512, activation='relu'),
    # Dropout(0.2),
    Dense(units=classes, activation='softmax'),
  ])

  model.compile(
      optimizer='adam',
      loss=loss,
      metrics=['accuracy'],
      )
  
  model.summary()
  return model


## Fitting a given model

In [ ]:
def train(model, dataset):
  model.fit(
    dataset.x_train,
    dataset.y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(dataset.x_valid, dataset.y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=PATIENCE_IN_EPOCHS)],
  )

def evaluate(model, x, y):
  score = model.evaluate(x, y, verbose=0)
  return score


# Experiments

## Regression

#### Predict hours and minutes using decimal representation and MAE

In [45]:
y_decimal_train = decimal_representation_of(y_train)
y_decimal_valid = decimal_representation_of(y_valid)
y_decimal_test = decimal_representation_of(y_test)

dataset = Dataset(
      name='decimal-representation',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_decimal_train,
      x_valid=x_valid,
      y_valid=y_decimal_valid,
      x_test=x_test,
      y_test=y_decimal_test,
  )

decimal_hours_minutes_model = regression_cnn_1('mae', dataset, 1, mean_minutes_loss_metric)
train(decimal_hours_minutes_model, dataset)
print(f'Minutes loss on test set: {evaluate(decimal_hours_minutes_model, dataset.x_test, dataset.y_test)[1]}')


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_7 (Rescaling)     (None, 150, 150, 1)       0         
                                                                 
 conv2d_35 (Conv2D)          (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 73, 73, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 71, 71, 32)        4640      
                                                                 
 conv2d_37 (Conv2D)          (None, 69, 69, 32)        9248      
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 34, 34, 32)       0         
 g2D)                                                 

##### Observations
Around 18 minutes loss on test set was observed when using decimal representation of time, and mean absolute error as a loss function.

Defining a custom loss which is closer to the common sense loss of the problem we might achieve a better

#### Predict hours and minutes using decimal representation and custom loss

In [47]:
y_decimal_train = decimal_representation_of(y_train)
y_decimal_valid = decimal_representation_of(y_valid)
y_decimal_test = decimal_representation_of(y_test)

dataset = Dataset(
      name='decimal-representation-common-sense-loss',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_decimal_train,
      x_valid=x_valid,
      y_valid=y_decimal_valid,
      x_test=x_test,
      y_test=y_decimal_test,
  )

decimal_hours_minutes_custom_loss_model = regression_cnn_1(DecimalTimesMeanLoss(), dataset, 1, mean_minutes_loss_metric)
train(decimal_hours_minutes_custom_loss_model, dataset)
print(f'Minutes loss on test set: {evaluate(decimal_hours_minutes_custom_loss_model, dataset.x_test, dataset.y_test)[1]}')


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_9 (Rescaling)     (None, 150, 150, 1)       0         
                                                                 
 conv2d_45 (Conv2D)          (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 73, 73, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_46 (Conv2D)          (None, 71, 71, 32)        4640      
                                                                 
 conv2d_47 (Conv2D)          (None, 69, 69, 32)        9248      
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 34, 34, 32)       0         
 g2D)                                                 

##### Observations
As expected using the same model with "common sense" loss function (the absolute value of the time difference between the predicted and the actual time), we achieved a better accuracy with 15.39 minutes mean loss.

#### Predict hours only




In [ ]:
y_hours_train = y_train[:, 0].astype('float32')
y_hours_valid = y_valid[:, 0].astype('float32')
y_hours_test = y_test[:, 0].astype('float32')

dataset = Dataset(
      name='hours-common-sense-loss',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_hours_train,
      x_valid=x_valid,
      y_valid=y_hours_valid,
      x_test=x_test,
      y_test=y_hours_test,
  )

hours_custom_loss_model = regression_cnn_1(DecimalTimesMeanLoss(), dataset, 1)
train(hours_custom_loss_model, dataset)
print(f'Hours loss on test set: {evaluate(hours_custom_loss_model, dataset.x_test, dataset.y_test)[1] / 60}')


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_9 (Rescaling)     (None, 150, 150, 1)       0         
                                                                 
 conv2d_45 (Conv2D)          (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 73, 73, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_46 (Conv2D)          (None, 71, 71, 32)        4640      
                                                                 
 conv2d_47 (Conv2D)          (None, 69, 69, 32)        9248      
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 34, 34, 32)       0         
 g2D)                                                 

##### Observations
Applying Regression only in hours with the common sense loss we achieved approximately 15 minutes difference between the actual and the predicted hours

#### Predict minutes only

In [ ]:
y_minutes_train = y_train[:, 1].astype('float32')
y_minutes_valid = y_valid[:, 1].astype('float32')
y_minutes_test = y_test[:, 1].astype('float32')

dataset = Dataset(
      name='minutes-common-sense-loss',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_minutes_train,
      x_valid=x_valid,
      y_valid=y_minutes_valid,
      x_test=x_test,
      y_test=y_minutes_test,
  )

minutes_custom_loss_model = regression_cnn_1(MinutesMeanLoss(), dataset, 1)
train(minutes_custom_loss_model, dataset)
print(f'Minutes loss on test set: {evaluate(minutes_custom_loss_model, dataset.x_test, dataset.y_test)}')


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_12 (Rescaling)    (None, 150, 150, 1)       0         
                                                                 
 conv2d_60 (Conv2D)          (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 73, 73, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_61 (Conv2D)          (None, 71, 71, 32)        4640      
                                                                 
 conv2d_62 (Conv2D)          (None, 69, 69, 32)        9248      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 34, 34, 32)       0         
 g2D)                                                

##### Observations
Predicting only the minutes by using the regression model with common sense loss,
less than 3 minutes loss was recorded on the test set. 


### Cyclical representation

In [34]:
def get_angle(sin, cos):
    angle = math.atan2(sin, cos) * 180 / math.pi # ALWAYS USE THIS
    
    if angle < 0: 
        angle += 360
    
    return angle


def get_minutes(sin, cos, max_value):
    
    return int(get_angle(sin, cos)*max_value*1.0/180)

#### Predict hours and minutes with cyclical representation (sine, cosine)

In [43]:

y_cyclical_train = cyclical_representation_of(y_train).T
y_cyclical_valid = cyclical_representation_of(y_valid).T
y_cyclical_test = cyclical_representation_of(y_test).T

print(y_cyclical_train.shape)
print(y_cyclical_train[0])
print(y_cyclical_valid.shape)
print(y_cyclical_test.shape)
print(x_train.shape)
# y_decimal_train.shape

dataset = Dataset(
      name='cyclical-representation',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_cyclical_train,
      x_valid=x_valid,
      y_valid=y_cyclical_valid,
      x_test=x_test,
      y_test=y_cyclical_test
  )

cyclical_time_custom_loss_model = regression_cnn_1('mae', dataset, 2)
train(cyclical_time_custom_loss_model, dataset)


(12240, 2)
[-0.9998477   0.01745241]
(2160, 2)
(3600, 2)
(12240, 150, 150, 1)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_6 (Rescaling)     (None, 150, 150, 1)       0         
                                                                 
 conv2d_30 (Conv2D)          (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 73, 73, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 71, 71, 32)        4640      
                                                                 
 conv2d_32 (Conv2D)          (None, 69, 69, 32)        9248      
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 34, 3

In [44]:
y_pred_cyclical = cyclical_time_custom_loss_model.predict(x_test)

adjusted_mae_numpy(
    np.array([get_minutes(sin, cos, 720) for (sin, cos) in y_pred_cyclical]).reshape(-1, 1), 
    np.array([get_minutes(sin, cos, 720) for (sin, cos) in y_cyclical_test]).reshape(-1, 1),
    720
    )

16.055

#### Predict hours with cyclical representation

In [48]:
y_cyclical_hours_train = cyclical_representation_of_hours(y_train[:,0]).T
y_cyclical_hours_valid = cyclical_representation_of_hours(y_valid[:,0]).T
y_cyclical_hours_test = cyclical_representation_of_hours(y_test[:,0]).T

dataset = Dataset(
      name='hours-cyclical-representation',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_cyclical_hours_train,
      x_valid=x_valid,
      y_valid=y_cyclical_hours_valid,
      x_test=x_test,
      y_test=y_cyclical_hours_test
  )


cyclical_hours_custom_loss_model = regression_cnn_1('mae', dataset, 2)
train(cyclical_hours_custom_loss_model, dataset)


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_10 (Rescaling)    (None, 150, 150, 1)       0         
                                                                 
 conv2d_50 (Conv2D)          (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 73, 73, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_51 (Conv2D)          (None, 71, 71, 32)        4640      
                                                                 
 conv2d_52 (Conv2D)          (None, 69, 69, 32)        9248      
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 34, 34, 32)       0         
 g2D)                                                

In [53]:
y_pred_cyclical = cyclical_hours_custom_loss_model.predict(x_test)
adjusted_mae_numpy(
    np.array([get_minutes(sin, cos, 12) for (sin, cos) in y_pred_cyclical]).reshape(-1, 1), 
    np.array([get_minutes(sin, cos, 12) for (sin, cos) in y_cyclical_test]).reshape(-1, 1),
    12
    )

1.2444444444444445

#### Predict minutes with cyclical representation

In [54]:
y_cyclical_minutes_train = cyclical_representation_of_minutes(y_train[:,1]).T
y_cyclical_minutes_valid = cyclical_representation_of_minutes(y_valid[:,1]).T
y_cyclical_minutes_test = cyclical_representation_of_minutes(y_test[:,1]).T

dataset = Dataset(
      name='minutes-cyclical-representation',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_cyclical_minutes_train,
      x_valid=x_valid,
      y_valid=y_cyclical_minutes_valid,
      x_test=x_test,
      y_test=y_cyclical_minutes_test
  )


cyclical_minutes_custom_loss_model = regression_cnn_1('mae', dataset, 2)
train(cyclical_minutes_custom_loss_model, dataset)


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_11 (Rescaling)    (None, 150, 150, 1)       0         
                                                                 
 conv2d_55 (Conv2D)          (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 73, 73, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_56 (Conv2D)          (None, 71, 71, 32)        4640      
                                                                 
 conv2d_57 (Conv2D)          (None, 69, 69, 32)        9248      
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 34, 34, 32)       0         
 g2D)                                                

In [57]:
y_pred_cyclical = cyclical_minutes_custom_loss_model.predict(x_test)

adjusted_mae_numpy(
    np.array([get_minutes(sin, cos, 60) for (sin, cos) in y_pred_cyclical]).reshape(-1, 1), 
    np.array([get_minutes(sin, cos, 60) for (sin, cos) in y_cyclical_test]).reshape(-1, 1),
    60
    )

15.920555555555556

## Classification

### Predict hours and minutes as 24 classes, one for each 30 minutes

In [ ]:
y_grouped_train = grouped_in_classes(y_train, 30)
y_grouped_test = grouped_in_classes(y_test, 30)
y_grouped_valid = grouped_in_classes(y_valid, 30)
print(y_grouped_train[0])
print(y_train[0])

dataset_24 = Dataset(
      name='classification',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_grouped_train,
      x_valid=x_valid,
      y_valid=y_grouped_valid,
      x_test=x_test,
      y_test=y_grouped_test
  )

classification_24_classes_model = classification_cnn(dataset_24, 24)
train(classification_24_classes_model, dataset_24)
print(f'Accuracy on test set: {evaluate(classification_24_classes_model, dataset_24.x_test, dataset_24.y_test)[1]}')


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[9 2]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 150, 150, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 73, 73, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 71, 71, 32)        4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 69, 69, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 34, 34, 32)       0         


In [ ]:
y_test_24 = np.argmax(dataset_24.y_test, axis=1).reshape(-1, 1)

y_pred_24 = classification_24_classes_model.predict(dataset_24.x_test)
y_pred_24 = np.argmax(y_pred_24, axis=1).reshape(-1, 1)

print(f"MAE: {mean_absolute_error(y_pred_24, y_test_24)}")
print(f"Adjusted MAE: {adjusted_mae_numpy(y_pred_24, y_test_24, 12)}")

MAE: 0.8811111111111111
Adjusted MAE: 0.5994444444444444


In [ ]:
print(classification_report(y_test_24, y_pred_24))

              precision    recall  f1-score   support

           0       0.73      0.93      0.82       155
           1       0.72      0.69      0.71       133
           2       0.82      0.64      0.72       159
           3       0.81      0.73      0.77       149
           4       0.72      0.72      0.72       158
           5       0.83      0.73      0.78       164
           6       0.76      0.59      0.66       134
           7       0.76      0.83      0.80       145
           8       0.65      0.71      0.68       149
           9       0.86      0.78      0.82       160
          10       0.76      0.85      0.80       143
          11       0.75      0.84      0.79       136
          12       0.78      0.86      0.82       162
          13       0.75      0.75      0.75       151
          14       0.92      0.90      0.91       157
          15       0.71      0.75      0.73       148
          16       0.83      0.84      0.84       167
          17       0.73    

### Predict hours and minutes as 72 classes, one for each 10 minutes

In [ ]:
y_grouped_train = grouped_in_classes(y_train, 10)
y_grouped_test = grouped_in_classes(y_test, 10)
y_grouped_valid = grouped_in_classes(y_valid, 10)
print(y_grouped_train[0])
print(y_train[0])

dataset_72 = Dataset(
      name='classification',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_grouped_train,
      x_valid=x_valid,
      y_valid=y_grouped_valid,
      x_test=x_test,
      y_test=y_grouped_test
  )

classification_72_classes_model = classification_cnn(dataset_72, 72)
train(classification_72_classes_model, dataset_72)
print(f'Accuracy on test set: {evaluate(classification_72_classes_model, dataset_72.x_test, dataset_72.y_test)[1]}')


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[9 2]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 150, 150, 1)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 73, 73, 16)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 71, 71, 32)        4640      
                                                                 
 conv2d_7 (Conv2D)           (None, 69, 69, 32)        9248      
                                 

In [ ]:
y_test_72 = np.argmax(dataset_72.y_test, axis=1).reshape(-1, 1)

y_pred_72 = classification_72_classes_model.predict(dataset_72.x_test)
y_pred_72 = np.argmax(y_pred_72, axis=1).reshape(-1, 1)

print(f"MAE: {mean_absolute_error(y_pred_72, y_test_72)}")
print(f"Adjusted MAE: {adjusted_mae_numpy(y_pred_72, y_test_72, 12)}")

MAE: 4.033333333333333
Adjusted MAE: 2.823888888888889


In [ ]:
print(classification_report(y_test_72, y_pred_72))

              precision    recall  f1-score   support

           0       0.65      0.64      0.64        66
           1       0.63      0.74      0.68        43
           2       0.74      0.70      0.72        46
           3       0.61      0.52      0.56        42
           4       0.72      0.62      0.67        50
           5       0.56      0.78      0.65        41
           6       0.64      0.66      0.65        44
           7       0.79      0.57      0.66        58
           8       0.72      0.68      0.70        57
           9       0.55      0.78      0.64        50
          10       0.57      0.76      0.65        46
          11       0.71      0.74      0.72        53
          12       0.56      0.76      0.64        49
          13       0.46      0.68      0.55        57
          14       0.76      0.48      0.59        52
          15       0.69      0.43      0.53        58
          16       0.63      0.45      0.52        49
          17       0.79    

### Predict hours as 12 classes

In [ ]:
encoder = OneHotEncoder(sparse=False)
y_hours_train = encoder.fit_transform([[yi] for yi in y_train[:,0]])
y_hours_valid = encoder.fit_transform([[yi] for yi in y_valid[:,0]])
y_hours_test = encoder.fit_transform([[yi] for yi in y_test[:,0]])

dataset_12 = Dataset(
      name='classification_hours',
      input_shape=input_shape,
      x_train=x_train,
      y_train=y_hours_train,
      x_valid=x_valid,
      y_valid=y_hours_valid,
      x_test=x_test,
      y_test=y_hours_test
  )

categorical_hours_model = classification_cnn(dataset_12, 12, 'categorical_crossentropy')
train(categorical_hours_model, dataset_12)
print(f'Accuracy on test set: {evaluate(categorical_hours_model, dataset_12.x_test, dataset_12.y_test)[1]}')


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_4 (Rescaling)     (None, 150, 150, 1)       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 146, 146, 16)      416       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 73, 73, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 71, 71, 32)        4640      
                                                                 
 conv2d_22 (Conv2D)          (None, 69, 69, 32)        9248      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 34, 34, 32)       0         
 g2D)                                                 

In [ ]:
y_test_12 = np.argmax(dataset_12.y_test, axis=1).reshape(-1, 1)

y_pred_12 = categorical_hours_model.predict(dataset_12.x_test)
y_pred_12 = np.argmax(y_pred_12, axis=1).reshape(-1, 1)

print(f"MAE: {mean_absolute_error(y_pred_12, y_test_12)}")
print(f"Adjusted MAE: {adjusted_mae_numpy(y_pred_12, y_test_12, 12)}")

MAE: 0.25083333333333335
Adjusted MAE: 0.1613888888888889


In [ ]:
print(classification_report(y_test_12, y_pred_12))

              precision    recall  f1-score   support

           0       0.91      0.88      0.89       288
           1       0.84      0.89      0.86       308
           2       0.87      0.82      0.84       322
           3       0.86      0.79      0.82       279
           4       0.83      0.88      0.85       309
           5       0.88      0.87      0.87       279
           6       0.86      0.90      0.88       313
           7       0.90      0.87      0.89       305
           8       0.85      0.88      0.87       320
           9       0.89      0.82      0.86       291
          10       0.85      0.90      0.87       281
          11       0.88      0.90      0.89       305

    accuracy                           0.87      3600
   macro avg       0.87      0.87      0.87      3600
weighted avg       0.87      0.87      0.87      3600



## Multi-head

In [ ]:
def get_y(y):

    y_hour = y[:, 0].reshape(-1, 1)
    y_minute = y[:, 1].reshape(-1, 1)

    y_hour = OneHotEncoder(sparse=False).fit_transform(y_hour)
    
    return [y_hour, y_minute]

X_train, X_test, y_train, y_test = train_test_split(x/255, y, test_size=0.2, random_state=42)

In [ ]:
def get_model(X):
    
    inp = Input(shape=(X.shape[1], X.shape[2], 1))

    # Convolutional Layers
    x = Conv2D(64, kernel_size=5, strides=2, activation='leaky_relu')(inp)
    x = MaxPooling2D(pool_size=(2, 2), strides=2)(x)
    x = Conv2D(64, kernel_size=3, strides=1, activation='leaky_relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, strides=1, activation='leaky_relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, strides=1, activation='leaky_relu')(x)
    x = Dropout(.4)(x)
    x = Flatten()(x)

    # Hour branch
    hour = Dense(256, activation='leaky_relu')(x)
    hour = Dense(256, activation='leaky_relu')(hour)
    hour = Dense(12, activation='softmax', name='hour')(hour)

    # Minute Branch
    minute = Dense(256, activation='leaky_relu')(x)
    minute = Dense(256, activation='leaky_relu')(minute)    
    minute = Dense(1, activation='linear', name='minute')(minute)

    model = Model(inputs=inp, outputs=[hour, minute])
    
    return model

model = get_model(x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 73, 73, 64)   1664        ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_3 (MaxPooling2D)  (None, 36, 36, 64)  0           ['conv2d_4[0][0]']               
                                                                                                  
 conv2d_5 (Conv2D)              (None, 34, 34, 64)   36928       ['max_pooling2d_3[0][0]']    

In [ ]:
model = get_model(x)

model.compile(loss=['categorical_crossentropy', 'mse'], optimizer='adam', metrics=['categorical_accuracy', 'mae'])

history = model.fit(x = X_train, y = get_y(y_train), batch_size = 32, epochs = 30, verbose = 1, validation_split = 0.2)

Epoch 1/30
360/360 [==============================] - 20s 10ms/step - loss: 336.3232 - hour_loss: 2.5176 - minute_loss: 333.8056 - hour_categorical_accuracy: 0.0878 - hour_mae: 0.1528 - minute_categorical_accuracy: 1.0000 - minute_mae: 15.5591 - val_loss: 301.9966 - val_hour_loss: 2.5027 - val_minute_loss: 299.4940 - val_hour_categorical_accuracy: 0.0837 - val_hour_mae: 0.1528 - val_minute_categorical_accuracy: 1.0000 - val_minute_mae: 14.9843
Epoch 2/30
360/360 [==============================] - 3s 9ms/step - loss: 316.8643 - hour_loss: 2.4956 - minute_loss: 314.3687 - hour_categorical_accuracy: 0.0833 - hour_mae: 0.1528 - minute_categorical_accuracy: 1.0000 - minute_mae: 15.2311 - val_loss: 300.0373 - val_hour_loss: 2.4907 - val_minute_loss: 297.5465 - val_hour_categorical_accuracy: 0.0847 - val_hour_mae: 0.1527 - val_minute_categorical_accuracy: 1.0000 - val_minute_mae: 14.9201
Epoch 3/30
360/360 [==============================] - 3s 9ms/step - loss: 293.9734 - hour_loss: 2.4794 - m

In [ ]:
[y_test_hour, y_test_minute] = get_y(y_test)

[y_pred_hour, y_pred_minute] = model.predict(X_test)

y_test_hour = np.argmax(y_test_hour, axis=1)

y_pred_hour = np.argmax(y_pred_hour, axis=1)

In [ ]:
print(classification_report(y_test_hour, y_pred_hour))

              precision    recall  f1-score   support

           0       0.85      0.82      0.83       288
           1       0.80      0.74      0.77       308
           2       0.78      0.86      0.82       322
           3       0.83      0.78      0.81       279
           4       0.79      0.83      0.81       309
           5       0.77      0.86      0.81       279
           6       0.88      0.73      0.80       313
           7       0.83      0.76      0.79       305
           8       0.73      0.92      0.82       320
           9       0.88      0.85      0.86       291
          10       0.81      0.88      0.84       281
          11       0.93      0.77      0.85       305

    accuracy                           0.82      3600
   macro avg       0.82      0.82      0.82      3600
weighted avg       0.82      0.82      0.82      3600



In [ ]:
print(f"MAE: {mean_absolute_error(y_pred_minute, y_test_minute)}")
print(f"Adjusted MAE: {adjusted_mae_numpy(y_pred_minute, y_test_minute, 60)}")

MAE: 3.4772036364509
Adjusted MAE: 3.089523071994384


In [ ]:
print(f"MAE: {mean_absolute_error(y_pred_hour, y_test_hour)}")
print(f"Adjusted MAE: {adjusted_mae_numpy(y_pred_hour.reshape(-1, 1), y_test_hour.reshape(-1, 1), 12)}")